In [1]:

import argparse

import time

from copy import deepcopy

from PIL import Image
import numpy as np
from tqdm import tqdm

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import pandas as pd
import os

try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
import torchvision.models as models

import clip
from clip.custom_clip import get_coop
from clip.cocoop import get_cocoop
from data.imagnet_prompts import imagenet_classes
from data.datautils import AugMixAugmenter, build_dataset
from utils.tools import Summary, AverageMeter, ProgressMeter, load_model_weight, set_random_seed, create_logger
from data.cls_to_names import *
from data.fewshot_datasets import fewshot_datasets
from data.imagenet_variants import thousand_k_to_200, imagenet_a_mask, imagenet_r_mask, imagenet_v_mask
from clip_retrieval.clip_client import ClipClient, Modality

from collections import defaultdict
%load_ext autoreload
%autoreload 2

client = ClipClient(
    url="http://127.0.0.1:1234/knn-service",
    indice_name='laion_400m',
    modality=Modality.IMAGE,
    num_images=10,
    deduplicate=False,
)
client_backup = ClipClient(
    url="http://127.0.0.1:1234/knn-service",
    indice_name='laion_400m',
    modality=Modality.IMAGE,
    num_images=200,
    deduplicate=False,
)

client_backup2 = ClipClient(
    url="http://127.0.0.1:1234/knn-service",
    indice_name='laion_400m',
    modality=Modality.IMAGE,
    num_images=1000,
    deduplicate=False,
)

## Class to names mapping
fewshot_datasets = ['DTD', 'Flower102', 'Food101', 'Cars', 'SUN397', 
                    'Aircraft', 'Pets', 'Caltech101', 'UCF101', 'eurosat']
test_sets = 'Caltech101'

/home/seongha/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def select_confident_samples(logits, top):
    batch_entropy = -(logits.softmax(1) * logits.log_softmax(1)).sum(1)
    idx = torch.argsort(batch_entropy, descending=False)[:int(batch_entropy.size()[0] * top)]
    return logits[idx], idx

def avg_entropy(outputs):
    # epsilon = 1e-10
    assert len(outputs) > 0
    assert torch.any(torch.isnan(outputs)) == False
    logits = outputs - outputs.logsumexp(dim=-1, keepdim=True) # logits = outputs.log_softmax(dim=1) [N, 1000]
    assert torch.any(torch.isnan(logits)) == False
    avg_logits = logits.logsumexp(dim=0) - np.log(logits.shape[0]) # avg_logits = logits.mean(0) [1, 1000]
    # print(avg_logits)
    if torch.any(torch.isnan(avg_logits)):
        print("average logits ", outputs.log_softmax(dim=1).mean(0))
    assert torch.any(torch.isnan(avg_logits)) == False
    
    min_real = torch.finfo(avg_logits.dtype).min
    avg_logits = torch.clamp(avg_logits, min=min_real)
    assert torch.any(torch.isnan(avg_logits)) == False
    return -((avg_logits) * (torch.exp(avg_logits))).sum(dim=-1)

In [3]:

def accuracy(output, target, topk=(1,), caption=None, logger=None):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        if output.shape[0] == 1:#only image prediction
            logit_k, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))
        else: # evaluate captions
            bag = []
            # # length = max(5, output.shape[0]-1)
            # cap_pred = output[1:]
            # # cap_pred = torch.mean(cap_pred, 0,  keepdim=True)
            # _, pred = cap_pred.topk(maxk, 1, True, True) #5, 1 #candidate labels
            # pred = pred.reshape(maxk, 1)
            pred = torch.mean(output, 0, keepdim=True)
            _, pred = pred.topk(maxk, 1, True, True)
            pred = pred.reshape(maxk, 1)
            correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            # if k == 1 and correct_k.item() == 0:

                # pred = pred.squeeze().tolist()
                # pred = [cls2names[lb] for lb in pred]

                # if logger: logger.info("wrong prediction, target {} & predicted value {}".format(target, pred))
            # elif k==1 and correct_k.item() == 1:
                # logger.info("wrong prediction , logit: ", output)
                # pred = pred.squeeze().tolist()
                # pred = [cls2names[lb] for lb in pred]

            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [4]:
#Parameters
tta_steps = 1
which_loss = "cosine"
gpu = 7
print_freq = 1000

In [5]:

def return_caption(img_path, retrieve_K=1):
    try:
        query_res = client.query(image=img_path)
        assert len(query_res) >= retrieve_K
        query_res = query_res[:retrieve_K]
        retrieved_txt= [D['caption'] for D in query_res]
        retrieved_url = [D['url'] for D in query_res]
        retrieved_score = [D['similarity'] for D in query_res]
        return retrieved_txt, retrieved_score
    except:
        query_res = client_backup2.query(image=img_path)
        if len(query_res) >= retrieve_K:
            query_res = query_res[:retrieve_K]
            retrieved_txt= [D['caption'] for D in query_res]
            retrieved_url = [D['url'] for D in query_res]
            retrieved_score = [D['similarity'] for D in query_res]
            return retrieved_txt, retrieved_score
        else:
            return None, None


                    

In [6]:
def test_time_adapt_eval_image(val_loader, model, model_state, optimizer, optim_state, scaler, save_result=None):
    batch_time = AverageMeter('Time', ':6.3f', Summary.NONE)
    top1 = AverageMeter('Acc@1', ':6.2f', Summary.AVERAGE)
    top2 = AverageMeter('Acc@2', ':6.2f', Summary.AVERAGE)
    top3 = AverageMeter('Acc@3', ':6.2f', Summary.AVERAGE)
    top4 = AverageMeter('Acc@4', ':6.2f', Summary.AVERAGE)
    top5 = AverageMeter('Acc@5', ':6.2f', Summary.AVERAGE)
    
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, top1, top2, top3, top4, top5],
        prefix='Test: ',
        logger = None)

    # reset model and switch to evaluate mode
    model.eval()
    with torch.no_grad():
        model.reset()
    end = time.time()
    cnt_empty = 0
    if save_result == None: save_result = defaultdict(list)
    
    for i, (image, target, imagepath) in tqdm(enumerate(val_loader)): 
        assert gpu is not None
        # print("Image Path ", imagepath[0])
        save_result['image_path'].append(imagepath[0])
        
        target = target.cuda(gpu, non_blocking=True)
        
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                #image
                output = model.inference(image.cuda(gpu, non_blocking=True))

        # output = output / output.norm(dim=-1, keepdim=True)
        save_result['image_entropy'].append('{:.4f}'.format(avg_entropy(output)))
        output = torch.nn.functional.softmax(output, dim=-1)
        logit_k, pred = output.topk(2, 1, True, True) #1,1
        
        pred = pred[:,0].t()
        logit_k= logit_k.squeeze()
        correct = pred.eq(target)
        # print(correct)
        correct = correct.reshape(-1).float().sum(0, keepdim=True).item() #1 or 0
        save_result['image_correct'].append(int(correct))
        save_result['image_logit'].append('{:.4f}'.format(logit_k[0].item()))
        if correct == 1:
            # correct label - top2
            # print(logit_k)
            save_result['image_gap'].append('{:.4f}'.format(logit_k[0].item() - logit_k[1].item()))
        else:
            # incorrect top1 - real label
            # print(target, output.squeeze()[target], logit_k )
            save_result['image_gap'].append('{:.4f}'.format(logit_k[0].item() - output.squeeze()[target].item()))
        
                
        acc1, acc2, acc3, acc4, acc5 = accuracy(output, target, topk=(1, 2, 3, 4, 5), caption=None, logger=None)
        # print(acc1, acc2, acc3, acc4, acc5)
        top1.update(acc1[0], image.size(0))
        top2.update(acc2[0], image.size(0))
        top3.update(acc3[0], image.size(0))
        top4.update(acc4[0], image.size(0))
        top5.update(acc5[0], image.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if (i+1) % print_freq == 0:
            progress.display(i)
        

    progress.display_summary()
    return [top1.avg, top2.avg, top3.avg, top4.avg, top5.avg], save_result

def test_time_adapt_eval_caption(val_loader, model, model_state, optimizer, optim_state, scaler, save_result=False):
    batch_time = AverageMeter('Time', ':6.3f', Summary.NONE)
    top1 = AverageMeter('Acc@1', ':6.2f', Summary.AVERAGE)
    top2 = AverageMeter('Acc@2', ':6.2f', Summary.AVERAGE)
    top3 = AverageMeter('Acc@3', ':6.2f', Summary.AVERAGE)
    top4 = AverageMeter('Acc@4', ':6.2f', Summary.AVERAGE)
    top5 = AverageMeter('Acc@5', ':6.2f', Summary.AVERAGE)
    
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, top1, top2, top3, top4, top5],
        prefix='Test: ',
        logger = None)

    # reset model and switch to evaluate mode
    model.eval()
    with torch.no_grad():
        model.reset()
    end = time.time()
    cnt_empty = 0
    assert save_result != None
    
    for i, (image, target, imagepath) in tqdm(enumerate(val_loader)): 
        assert gpu is not None
        # print("Image Path ", imagepath[0])
        # save_result['image_path'] = imagepath[0]
        
        target = target.cuda(gpu, non_blocking=True)
        
        with torch.no_grad():
            with torch.cuda.amp.autocast():
                #caption
                retrieved_Caption, _ = return_caption(imagepath[0])
                if retrieved_Caption==None:
                    cnt_empty +=1
                    continue
                save_result['caption'].append(retrieved_Caption[0])
                output_caption = model.caption_ensemble(retrieved_Caption)

        # output_caption = output_caption / output_caption.norm(dim=-1, keepdim=True)
        save_result['caption_entropy'].append('{:.4f}'.format(avg_entropy(output_caption)))
        output_caption = torch.nn.functional.softmax(output_caption, dim=-1)
        logit_k, pred = output_caption.topk(2, 1, True, True) #1,2
        logit_k = logit_k.squeeze()
        
        pred = pred[:,0].t()
        correct = pred.eq(target)
        correct = correct.reshape(-1).float().sum(0, keepdim=True).item() #1 or 0
        save_result['caption_correct'].append(int(correct))
        save_result['caption_logit'].append('{:.4f}'.format(logit_k[0].item()))
        if correct == 1:
            # correct label - top2
            # print(logit_k)
            save_result['caption_gap'].append('{:.4f}'.format(logit_k[0].item() - logit_k[1].item()))
        else:
            # incorrect top1 - real label
            # print(target, output.squeeze()[target], logit_k )
            save_result['caption_gap'].append('{:.4f}'.format(logit_k[0].item() - output_caption.squeeze()[target].item()))
        
         
        acc1, acc2, acc3, acc4, acc5 = accuracy(output_caption, target, topk=(1, 2, 3, 4, 5), caption=None, logger=None)
        # print(acc1, acc2, acc3, acc4, acc5)
        top1.update(acc1[0], image.size(0))
        top2.update(acc2[0], image.size(0))
        top3.update(acc3[0], image.size(0))
        top4.update(acc4[0], image.size(0))
        top5.update(acc5[0], image.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if (i+1) % print_freq == 0:
            progress.display(i)
    
    print("empty caption count = {}".format(cnt_empty))
    progress.display_summary()
    return [top1.avg, top2.avg, top3.avg, top4.avg, top5.avg], save_result

In [7]:
##parameters
fewshot_datasets = ['DTD', 'Flower102', 'Food101', 'Cars', 'SUN397', 
                    'Aircraft', 'Pets', 'Caltech101', 'UCF101', 'eurosat']
arch='ViT-B/16'
n_ctx=4
ctx_init="a_photo_of_a"
lr = 5e-3

In [8]:
# load model
if test_sets in fewshot_datasets:
    classnames = eval("{}_classes".format(test_sets.lower()))
model = get_coop(arch, test_sets, gpu, n_ctx, ctx_init)
model_state = None

cross_check = set()
for name, param in model.named_parameters():

    if "prompt_learner" not in name:
        param.requires_grad_(False)
    if param.requires_grad : cross_check.add(name)
print("tuing parameters ", cross_check)

print("=> Model created: visual backbone {}".format(arch))

assert gpu is not None
torch.cuda.set_device(gpu)
model = model.cuda(gpu)

trainable_param = model.prompt_learner.parameters()
optimizer = torch.optim.AdamW(trainable_param, lr)
optim_state = deepcopy(optimizer.state_dict())

# setup automatic mixed-precision (Amp) loss scaling
scaler = torch.cuda.amp.GradScaler(init_scale=1000)

print('=> Using native Torch AMP. Training in mixed precision.')

cudnn.benchmark = True

Initializing the contect with given words: [a_photo_of_a]
Initial context: "a photo of a"
Number of context words (tokens): 4
tuing parameters  {'prompt_learner.ctx'}
=> Model created: visual backbone ViT-B/16
=> Using native Torch AMP. Training in mixed precision.


In [9]:
resolution = 224
workers = 4
dataset_mode = 'test'
data = '/data/seongha'
import sys
from collections import defaultdict
    # norm stats from clip.load()
normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                     std=[0.26862954, 0.26130258, 0.27577711])
    
    # iterating through eval datasets
datasets = test_sets.split("/")
results = {}

# with open('inferece_image_caption{}.txt'.format(test_sets), 'w') as f:
    # sys.stdout = f


for set_id in datasets:
    Dict = defaultdict(list)
    data_transform = transforms.Compose([
        transforms.Resize(resolution, interpolation=BICUBIC),
        transforms.CenterCrop(resolution),
        transforms.ToTensor(),
        normalize,
    ])
    batchsize = 1
    print("evaluating: {}".format(set_id))
    classnames = eval("{}_classes".format(set_id.lower()))
    model.reset_classnames(classnames, arch)

    val_dataset = build_dataset(set_id, data_transform, data, mode=dataset_mode)
    total_length = len(val_dataset)
    print("number of test samples: {}".format(len(val_dataset)))

    val_loader = torch.utils.data.DataLoader(
                val_dataset,
                batch_size=batchsize, shuffle=False,
                num_workers=workers, pin_memory=True)
        
    results['image'], tmp = test_time_adapt_eval_image(val_loader, model, model_state, optimizer, optim_state, scaler, Dict)
    Dict = tmp
    results['caption'], tmp = test_time_adapt_eval_caption(val_loader, model, model_state, optimizer, optim_state, scaler, Dict)
    Dict = tmp
    # assert len(Dict['image_path']) == len(Dict['caption']) and len(Dict['image_correct']) == len(Dict['caption_correct']), [len(v) for k, v in Dict.items()]
    del val_dataset, val_loader

    # try:
    #     print("=> Acc. on testset [{}]: @1 {}/ @2 {}/ @3 {}/ @4 {}/ @5 {}".format(set_id, results[set_id][0], results[set_id][1], results['image'][2], results['image'][3], results['image'[4], results['image'][5]]))
    # except:
    #     print("=> Acc. on testset [{}]: {}".format(set_id, results[set_id]))
# sys.stdout = sys.__stdout__
    tmp = {k: v for k, v in Dict.items() if "image" in k}
    df_img = pd.DataFrame(tmp)
    df_img = df_img.reset_index()

    tmp = {k: v for k, v in Dict.items() if "caption" in k}
    df_cap = pd.DataFrame(tmp)
    df_cap = df_cap.reset_index()

    path = './notebook/inference_image_caption'
    os.makedirs(path, exist_ok=True)
    # df.to_csv(os.path.join(path, 'inference_image_caption_{}.csv'.format(set_id)))

    with open(os.path.join(path,'inference_image_caption_{}.txt'.format(set_id)), 'w') as f:
        
        img_corr_ind = df_img.loc[df_img['image_correct'] == 1, 'index'].to_list()
        cap_corr_ind = df_cap.loc[df_cap['caption_correct'] == 1, 'index'].to_list()
        neither = set(df_img.loc[df_img['image_correct'] == 0, 'index'].to_list()) & set(df_cap.loc[df_cap['caption_correct'] == 0, 'index'].to_list())
        #image accuracy, caption ensemble accuracy
        f.write("1. Image accuracy {:.4f}, Caption Accuracy {:.4f}\n".format( len(img_corr_ind)/total_length, len(cap_corr_ind)/total_length))
        # 4 cases
        union = set(img_corr_ind) | set(cap_corr_ind)
        f.write("2: 4 cases\n")
        f.write("Union :{}\n".format( len(union)))
        intersection = set(img_corr_ind) & set(cap_corr_ind)
        f.write("intersection {}\n".format( len(intersection)))
        img_diff = set(img_corr_ind) - set(cap_corr_ind)
        f.write("image only {}\n".format(len(img_diff)))
        cap_diff = set(cap_corr_ind) - set(img_corr_ind)
        f.write("cap only {}\n".format( len(cap_diff)))
        f.write("neither {}\n".format( len(neither)))
        # max accuracy
        f.write("3. Max accuracy\n")
        f.write("Max accuracy: {:.4f}\n".format(len(union)/total_length*100)) 
        #entropy, logit gap
        f.write("4. Entropy & Logit Gap\n")
        f.write("Image\n")
        img_correct = df_img.loc[df_img['image_correct'] == 1]
        f.write("correct\n")
        f.write(" {}\n".format(str(img_correct.shape)))
        f.write("top1 - top2 mean, std\n")
        f.write("{} {}\n".format(img_correct['image_gap'].astype(float).mean(), img_correct['image_gap'].astype(float).std() ))
        f.write("Entropy mean {}\n".format(str(img_correct['image_entropy'].astype(float).mean())))
        f.write("")
        img_wrong = df_img.loc[df_img['image_correct'] == 0]
        f.write("wrong\n")
        f.write(" {}\n".format(str(img_wrong.shape)))
        f.write("pred(top1) - target mean, std\n")
        f.write("{} {}\n".format(img_wrong['image_gap'].astype(float).mean(), img_wrong['image_gap'].astype(float).std() ))
        f.write("Entropy mean {}\n".format(str(img_wrong['image_entropy'].astype(float).mean())))
        f.write("-"*10 + "\n")
        
        f.write("Caption\n")
        cap_correct = df_cap.loc[df_cap['caption_correct'] == 1]
        f.write("correct\n")
        f.write(" {}\n".format(str(cap_correct.shape)))
        f.write("top1 - top2 mean, std\n")
        f.write("{} {}\n".format(cap_correct['caption_gap'].astype(float).mean(), cap_correct['caption_gap'].astype(float).std() ))
        f.write("Entropy mean {}\n".format(str(cap_correct['caption_entropy'].astype(float).mean())))
        f.write("")
        cap_wrong = df_cap.loc[df_cap['caption_correct'] == 0]
        f.write("wrong\n")
        f.write(" {}\n".format(str(cap_wrong.shape)))
        f.write("pred(top1) - target mean, std\n")
        f.write("{} {}\n".format(cap_wrong['caption_gap'].astype(float).mean(), cap_wrong['caption_gap'].astype(float).std() ))
        f.write("Entropy mean {}\n".format(str(cap_wrong['caption_entropy'].astype(float).mean())))
        

evaluating: Caltech101
number of test samples: 2465


1001it [01:06, 20.00it/s]

Test: [ 999/2465]	Time  0.033 ( 0.067)	Acc@1   0.00 ( 95.70)	Acc@2 100.00 ( 98.60)	Acc@3 100.00 ( 98.90)	Acc@4 100.00 ( 99.40)	Acc@5 100.00 ( 99.80)


2005it [02:06, 18.99it/s]

Test: [1999/2465]	Time  0.064 ( 0.063)	Acc@1 100.00 ( 92.65)	Acc@2 100.00 ( 98.55)	Acc@3 100.00 ( 99.15)	Acc@4 100.00 ( 99.45)	Acc@5 100.00 ( 99.75)


...

Test: [ 999/2465]	Time  0.285 ( 0.344)	Acc@1   0.00 ( 83.70)	Acc@2 100.00 ( 89.80)	Acc@3 100.00 ( 92.00)	Acc@4 100.00 ( 93.40)	Acc@5 100.00 ( 94.20)


2000it [11:37,  3.23it/s]

Test: [1999/2465]	Time  0.289 ( 0.349)	Acc@1 100.00 ( 75.40)	Acc@2 100.00 ( 84.95)	Acc@3 100.00 ( 87.60)	Acc@4 100.00 ( 88.75)	Acc@5 100.00 ( 89.55)


2465it [14:02,  2.92it/s]

empty caption count = 0
 *  Acc@1 75.091 Acc@2 84.381 Acc@3 86.775 Acc@4 87.951 Acc@5 88.641


In [21]:
df = pd.DataFrame(Dict)
for i_set, i_name in zip([img_diff, cap_diff, intersection, neither, union], ['img_diff', 'cap_diff', 'intersection', 'neither', 'union']):
    df_ = df.iloc[list(i_set)]
    print(i_name, df_.shape)
    print("Image Entropy mean {}".format(str(df_['image_entropy'].astype(float).mean())))
    print("Caption Entropy mean {}\n".format(str(df_['caption_entropy'].astype(float).mean())))

img_diff (508, 10)
Image Entropy mean 0.6352336614173227
Caption Entropy mean 1.1352950787401574

cap_diff (70, 10)
Image Entropy mean 1.0092857142857141
Caption Entropy mean 0.3035428571428571

intersection (1782, 10)
Image Entropy mean 0.3845969135802469
Caption Entropy mean 0.18090819304152636

neither (105, 10)
Image Entropy mean 1.1489142857142858
Caption Entropy mean 0.7517161904761906

union (2360, 10)
Image Entropy mean 0.4570764406779661
Caption Entropy mean 0.38998148305084746

